In [1]:
# keri.core.coring.Saider._derive: ==> sad[label] = clas.Dummy * Matter.Sizes[code].fs

# the above appears insufficient to calculate nested saids.
# need something to recurse deep-first-search of saids and then apply: `sad[label] = clas.Dummy * Matter.Sizes[code].fs`
## then need to call return (Diger._digest(ser=cpa, code=code), sad) on the path with the label ( the nested structure )
## then finally calculate the said on the compact version.

## Something like below will replace and track the paths to process the above in the correct order?


def replace_label_with_said_and_track_paths(obj, label, debug=False):
    """
    Recursively replaces the specified label's value with a computed SAID in a nested dictionary structure,
    and tracks the paths to each label instance in depth-first order.

    Parameters:
        obj (dict): The input dictionary to process.
        label (str): The field name whose value should be replaced by the computed SAID.
        debug (bool): Flag for debugging output.

    Returns:
        dict: The updated dictionary with computed SAIDs replacing the label values.
        list: A list of paths to each label instance, ordered by depth (deepest first).
    """
    paths = []  # List to store paths to each label instance

    def recursive_replace(data, path=""):
        if isinstance(data, dict):
            updated_dict = {}
            for key, value in data.items():
                current_path = f"{path}.{key}" if path else key
                if key == label:
                    # Replace the label value with a computed SAID (placeholder here for demonstration)
                    dummy_matter = len(value) * '#'  # Replace with actual `get_blake3_256_said` function
                    updated_dict[key] = dummy_matter
                    if debug:
                        print(f"dummy_matter SAID for {current_path}: {dummy_matter}")

                    # Add path to paths list
                    paths.append(current_path)
                else:
                    # Recursively process nested dictionaries or lists
                    updated_dict[key] = recursive_replace(value, current_path)
            return updated_dict
        elif isinstance(data, list):
            return [recursive_replace(item, f"{path}[{i}]") for i, item in enumerate(data)]
        else:
            return data

    # Start recursive replacement
    updated_data = recursive_replace(obj)

    # Sort paths by depth (deepest paths first)
    paths.sort(key=lambda x: x.count('.'), reverse=True)

    return updated_data, paths

In [2]:
# EXAMPLE ACDC with deep nested said fields:
example1 = {'v': 'ACDC10JSON00084a_',
 'd': 'EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3',
 'i': 'EKXPX7hWw8KK5Y_Mxs2TOuCrGdN45vPIZ78NofRlVBws',
 'ri': 'EuqwB_iOD86eK0ynAhA6AYwWvPeBhvmbcmOD-9cCmiVU',
 's': 'ELG17Q0M-uLZcjidzVbF7KBkoUhZa1ie3Az3Q_8aYi8s',
 'a': {'d': 'E9-86Jag34CrJpfNFz_-7E5HA0Dj0FvcYNoFVe7qwkiI',
  'dt': '2022-08-25T14:07:30.536257+00:00',
  'i': 'EY4ldIBDZP4Tpnm3RX320BO0yz8Uz2nUSN-C409GnCJM',
  'AID': 'Esf8b_AngI1d0KbOFjPGIfpVani0HTagWeaYTLs14PlE',
  'LEI': '6383001AJTYIGC8Y1X37',
  'personLegalName': 'John Smith',
  'engagementContextRole': 'Chief Executive Officer',
  'D': {'d': 'EsOf5_YgX_64z4YuHNFWLUnIKcyvsVQOe_vJ_638X6gE',
   'le': {'n': 'ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs',
    's': 'ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY'}}},
 'e': {'d': 'EsOf5_YgX_64z4YuHNFWLUnIKcyvsVQOe_vJ_638X6gE',
  'le': {'n': 'ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs',
   's': 'ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY'}},
 'r': {'d': 'EDIai3Wkd-Z_4cezz9nYEcCK3KNH5saLvZoS_84JL6NU',
  'usageDisclaimer': {'l': 'Usage of a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, does not assert that the Legal Entity is trustworthy, honest, reputable in its business dealings, safe to do business with, or compliant with any laws or that an implied or expressly intended purpose will be fulfilled.'},
  'issuanceDisclaimer': {'l': 'All information in a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, is accurate as of the date the validation process was complete. The vLEI Credential has been issued to the legal entity or person named in the vLEI Credential as the subject; and the qualified vLEI Issuer exercised reasonable care to perform the validation process set forth in the vLEI Ecosystem Governance Framework.'},
  'privacyDisclaimer': {'l': 'Privacy Considerations are applicable to QVI ECR AUTH vLEI Credentials.  It is the sole responsibility of QVIs as Issuees of QVI ECR AUTH vLEI Credentials to present these Credentials in a privacy-preserving manner using the mechanisms provided in the Issuance and Presentation Exchange (IPEX) protocol specification and the Authentic Chained Data Container (ACDC) specification.  https://github.com/WebOfTrust/IETF-IPEX and https://github.com/trustoverip/tswg-acdc-specification.'}}}

In [3]:
replaced, said_paths = replace_label_with_said_and_track_paths(example1, 'd', True)


dummy_matter SAID for d: ############################################
dummy_matter SAID for a.d: ############################################
dummy_matter SAID for a.D.d: ############################################
dummy_matter SAID for e.d: ############################################
dummy_matter SAID for r.d: ############################################


In [4]:
## this should show the order and paths that need the saids to be computed
# - first path  a.D --> compute said of this struct and put in a.D.d
# - etc.
said_paths

['a.D.d', 'a.d', 'e.d', 'r.d', 'd']

In [5]:
# something like this could be used to collapse the paths to compute the saids of lower layers:
def collapse(obj, label):
    """
    Recursively collapses nested dictionaries to a specified label if it exists.

    Parameters:
        obj (dict): The dictionary to collapse.
        label (str): The field name to collapse on (e.g., 'id').

    Returns:
        dict: The collapsed dictionary.
    """
    collapsed_obj = {}

    for key, value in obj.items():
        if isinstance(value, dict) and label in value:
            # Collapse to the specified label's value if it exists in the nested dictionary
            collapsed_obj[key] = value[label]
        else:
            # Otherwise, keep the value as-is
            collapsed_obj[key] = value

    return collapsed_obj

In [6]:
def calc_collapsed_saids(upd, paths):
    o = upd.copy()
    for p in paths:
        path = p.split('.')
        obj = o.copy()
        for ki in range(len(path)-1):
            obj = obj[path[ki]]

       
        print(path)
        collapsed = collapse(obj, path[-1])
        # CALCULATE THE SAID ON THIS and put value in collapsed[path-1]  ==> obj
        # Diger._digest(ser=cpa, code=code), sad)
        print(collapsed)
        

calc_collapsed_saids(replaced, said_paths)

['a', 'D', 'd']
{'d': '############################################', 'le': {'n': 'ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs', 's': 'ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY'}}
['a', 'd']
{'d': '############################################', 'dt': '2022-08-25T14:07:30.536257+00:00', 'i': 'EY4ldIBDZP4Tpnm3RX320BO0yz8Uz2nUSN-C409GnCJM', 'AID': 'Esf8b_AngI1d0KbOFjPGIfpVani0HTagWeaYTLs14PlE', 'LEI': '6383001AJTYIGC8Y1X37', 'personLegalName': 'John Smith', 'engagementContextRole': 'Chief Executive Officer', 'D': '############################################'}
['e', 'd']
{'d': '############################################', 'le': {'n': 'ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs', 's': 'ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY'}}
['r', 'd']
{'d': '############################################', 'usageDisclaimer': {'l': 'Usage of a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, does not assert that the Legal Entity i

In [1]:
test ={
    "v": "ACDC10JSON00084a_",
    "d": "EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3",
    "i": "EKXPX7hWw8KK5Y_Mxs2TOuCrGdN45vPIZ78NofRlVBws",
    "ri": "EuqwB_iOD86eK0ynAhA6AYwWvPeBhvmbcmOD-9cCmiVU",
    "s": "ELG17Q0M-uLZcjidzVbF7KBkoUhZa1ie3Az3Q_8aYi8s",
    "a": {
        "d": "E9-86Jag34CrJpfNFz_-7E5HA0Dj0FvcYNoFVe7qwkiI",
        "dt": "2022-08-25T14:07:30.536257+00:00",
        "i": "EY4ldIBDZP4Tpnm3RX320BO0yz8Uz2nUSN-C409GnCJM",
        "AID": "Esf8b_AngI1d0KbOFjPGIfpVani0HTagWeaYTLs14PlE",
        "LEI": "6383001AJTYIGC8Y1X37",
        "personLegalName": "John Smith",
        "engagementContextRole": "Chief Executive Officer"
    },
    "e": {
        "d": "EsOf5_YgX_64z4YuHNFWLUnIKcyvsVQOe_vJ_638X6gE",
        "le": {
            "n": "ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs",
            "s": "ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY"
        }
    },
    "r": {
        "d": "EDIai3Wkd-Z_4cezz9nYEcCK3KNH5saLvZoS_84JL6NU",
        "usageDisclaimer": {
            "l": "Usage of a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, does not assert that the Legal Entity is trustworthy, honest, reputable in its business dealings, safe to do business with, or compliant with any laws or that an implied or expressly intended purpose will be fulfilled."
        },
        "issuanceDisclaimer": {
            "l": "All information in a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, is accurate as of the date the validation process was complete. The vLEI Credential has been issued to the legal entity or person named in the vLEI Credential as the subject; and the qualified vLEI Issuer exercised reasonable care to perform the validation process set forth in the vLEI Ecosystem Governance Framework."
        },
        "privacyDisclaimer": {
            "l": "Privacy Considerations are applicable to QVI ECR AUTH vLEI Credentials.  It is the sole responsibility of QVIs as Issuees of QVI ECR AUTH vLEI Credentials to present these Credentials in a privacy-preserving manner using the mechanisms provided in the Issuance and Presentation Exchange (IPEX) protocol specification and the Authentic Chained Data Container (ACDC) specification.  https://github.com/WebOfTrust/IETF-IPEX and https://github.com/trustoverip/tswg-acdc-specification."
        }
    }
}

In [2]:
pwd

'/media/dhash/s_main/hub/keri_stuff/simple-said/notebooks'

In [9]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

base_path = os.getcwd()
relative_path = os.path.abspath(os.path.join(base_path, '../src'))
sys.path.append(relative_path)

import utils as u

In [ ]:
u.get_

In [ ]:
u.get_blake3_256_said(

In [21]:
import secrets
import hashlib
import base64

def get_entropy(n=1024):
    """
    Get n bytes of entropy from multiple sources.

    :param n: The number of bytes to generate (default: 1024)
    :return: A bytes object containing the generated entropy
    """
    # Combine entropy from different sources:
    #   - /dev/urandom on Unix-like systems
    #   - os.urandom() on Windows and other platforms
    #   - time.time_ns() to add some extra randomness

    combined_entropy = bytearray(n)
    if hasattr(os, 'getpid'):  # Unix-like systems
        urandom_entropy = os.urandom(n // 2)  # /dev/urandom
        for i in range(len(combined_entropy)):
            combined_entropy[i] ^= urandom_entropy[i % (len(urandom_entropy))]
    elif hasattr(os, 'random'):  # Windows and other platforms
        random_entropy = os.urandom(n // 4)
        time_entropy = int(time.time_ns()).to_bytes(8, byteorder='big')
        for i in range(len(combined_entropy)):
            combined_entropy[i] ^= random_entropy[i % (len(random_entropy))]
            combined_entropy[i] ^= time_entropy[(i * 2) % len(time_entropy)]
    else:
        # Fall back to a simple PRNG
        prng = os.urandom(n)
        for i in range(len(combined_entropy)):
            combined_entropy[i] = prng[i % len(prng)]

    return bytes(combined_entropy)


def entropy_256():
    # Collect multiple sources of entropy
    entropy_source1 = secrets.token_bytes(32)
    entropy_source2 = os.urandom(32)
    entropy_bytes = get_entropy()
    combined_entropy = hashlib.sha256(entropy_source1 + entropy_source2+ entropy_bytes).digest()
    return combined_entropy

def bytes_to_base64url(bytes_data, strip_end = True):
    if strip_end:
        return base64.urlsafe_b64encode(bytes_data).rstrip(b'=').decode('utf-8')
    return  base64.urlsafe_b64encode(bytes_data).decode('utf-8')


'Dh1YaizLQbgvXEzIyy439KTs8pvYZQg-Gf1G-cjhAtI'

In [27]:
for i in range(10):
    e = entropy_256()
    print(bytes_to_base64url(e))

okkkhu0aEJPbbqMjVvff3PW0TPtIfYSmDT8TWh-dHfo
ZpYgQTJ0RLx8qeWeh58jK9wGnFLHXpNXcuS1DysGauA
CKEyE4Pgr6N2ZxcckR4YdbYn6DDgv94Hy7qYnUZw9Go
pkfkGBQIdPpbp_62ILJOoZjcCtk51A_2dGK0Wg1B5VQ
e44aE5SLdai9f7dw6-SSWiQnkyTpQ6WrDCphCbIsGv4
yNel1GW39H6H1ejWdO2yX6ZaoBK-jE38bpIvdSK8F4g
M5-cd-KlCZ4t5dNnzTNxDDgkWwYTwpM65dqzbbl89VM
ePrrz6AedfMye8_CJ9q-6BCwZGpdBSOb4fEx0MFJQ74
CbGofuUMNQP0zqq_e0iCshWAbEW_YuuIbW2Jwmb7M_0
k3CbqdW9AAwKsA09gfWRDVqY2CmXxZ67XH28KD2c978
